#### Data Extraction
extract essential information from JSON for the analysis
"title","views", "likes","comments", 'duration','tag','description', 'channel', 'category', 'published'

In [72]:
#extracting json to df
import os, json
import pandas as pd
import numpy as np
import glob
from datetime import datetime
pd.set_option('display.max_columns', None)

dir = 'data'
path = os.path.join(dir, '**/*.json')
file_list = glob.glob(path)
print('Total number of channels ' + str(len(file_list)))
# print(file_list)
dfs = list()

for file in file_list:
    with open(file, 'r') as f:
        data = json.load(f)
    channel_id, stats = data.popitem()
    pchannel_stats = stats["channel_statistics"]
    video_stats = stats["video_data"]
    # sorted_vids = sorted(video_stats.items(), key=lambda item: int(item[1]["viewCount"]),reverse=True)
    vids = video_stats.items()
    stats = []
    for vid in vids:
        video_id = vid[0]
        title = vid[1]["title"]
        try:
            views = vid[1]["viewCount"]
            likes = vid[1]["likeCount"]
            duration = vid[1]['duration']
            tags = vid[1]['tags']
            description = vid[1]['description']
            comments = vid[1]["commentCount"]
            channel = vid[1]['channelTitle']
            published = vid[1]['publishedAt'].split('T')[0]
        except:
            pass
        cat = os.path.dirname(file).split('\\')[1]
        stats.append([title,views, published, likes, comments, duration, tags, description, channel, cat])
    vid_df = pd.DataFrame(stats, columns=["title","views", 'published',"likes","comments", 'duration','tag','description', 'channel', 'category'])
    dfs.append(vid_df)
df = pd.concat(dfs, ignore_index=True)


Total number of channels 51


In [75]:
df.published = pd.to_datetime(df.published, format='%Y-%m-%d')
df.sample(5)

,title,views,published,likes,comments,duration,tag,description,channel,category
8188,The Mecca Mystery: Are Muslims Praying In The ...,9589268,2019-01-26,55494,21706,PT1H25M9S,"[petra islam, islamic documentaries in english...",The Sacred City presents compelling evidence t...,Timeline - World History Documentaries,History
11815,ASTIG: Pinay wagi sa Google contest | Bandila,182752,2019-05-03,2769,352,PT2M36S,"[ABS-CBN, ABS-CBN News, ABS-CBN Philippines, P...",Wagi ang isang 17 anyos na Pinay sa internatio...,ABS-CBN News,News
9930,Nora En Pure & Redondo - I Got To Do (Official...,945303,2016-02-22,14718,282,PT3M35S,"[Spinnin', Records, Nora En Pure, Redondo, I G...",Nora En Pure & Redondo - I Got To Do is OUT NO...,Spinnin' Records,Music
16467,Expedia Find Yours NFFTY Contest: Find Some Time,4434,2012-07-16,18,1,PT3M1S,"[Find Yours, Expedia, NFFTY]","For the launch of the new Expedia campaign, Fi...",Expedia,Travel
17619,What It Takes To Be A Rockette In Radio City's...,579682,2018-12-21,7646,311,PT4M26S,"[INSIDER, the rockettes, rockettes, christmas ...",The Radio City Rockettes have been American ic...,Insider,Travel
